## BERT 모델을 통한 자연어 분석

파일 확인

In [1]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('./data/emotion/') :
    for filename in filenames :
        print(os.path.join(dirname, filename))

./data/emotion/test.txt
./data/emotion/train.txt
./data/emotion/val.txt


In [2]:
#!pip install transformers

모델 불러오기

In [3]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
from transformers import BertTokenizer, BertConfig, AdamW, BertForSequenceClassification, get_linear_schedule_with_warmup

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, matthews_corrcoef

from tqdm import tqdm, trange, tnrange, tqdm_notebook
import random
import io

In [4]:
# torch device 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [5]:
# 무작위 값 지정
SEED = 19

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [6]:
# 각각의 데이터 불러오기
df_train = pd.read_csv("./data./emotion/train.txt", delimiter = ';', header=None, names = ['sentence', 'label'])
df_test = pd.read_csv("./data./emotion/test.txt", delimiter = ';', header=None, names = ['sentence', 'label'])
df_val = pd.read_csv("./data./emotion/val.txt", delimiter = ';', header=None, names = ['sentence', 'label'])


In [7]:
# 데이터 다 합치기
df = pd.concat([df_train, df_test, df_val])

In [8]:
# 라벨의 종류 확인
df['label'].unique()

array(['sadness', 'anger', 'love', 'surprise', 'fear', 'joy'],
      dtype=object)

In [9]:
# 라벨을 숫자로 바꾼 열 추가
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df['label_enc'] = labelencoder.fit_transform(df['label'])

In [10]:
# 생성된 열의 종류별로 확인
df[['label', 'label_enc']].drop_duplicates(keep='first')

,label,label_enc
0,sadness,4
2,anger,0
3,love,3
6,surprise,5
7,fear,1
8,joy,2


In [11]:
# 컬럼명 변경
df.rename(columns={'label' : 'label_desc'}, inplace = True)
df.rename(columns={'label_enc' : 'label'}, inplace = True)

In [12]:
# sentences 열 생성
sentences = df.sentence.values
# 라벨 저장
labels = df.label.values

In [13]:
# 라벨에 따른 데이터 분포 확인
print("Distribution of data based on labels :", df.label.value_counts())

Distribution of data based on labels : 2    6761
4    5797
0    2709
1    2373
3    1641
5     719
Name: label, dtype: int64


In [14]:
# 최대 길이 지정
MAX_LEN = 256

In [16]:
# 텍스트를 token화 해주는 BERT 활용
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)
input_ids = [tokenizer.encode(sent, add_special_Token = True, max_length = MAX_LEN, pad_to_max_length = True) for sent in sentences]

OSError: Windows requires Developer Mode to be activated, or to run Python as an administrator, in order to create symlinks.
In order to activate Developer Mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development